<a href="https://colab.research.google.com/github/pmusau17/AAF1Tenth/blob/master/colab-notebooks/only_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#libraries
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras
from keras import backend as K
import cv2
import os
import numpy as np
import sys
import csv
import glob
import time
from sklearn.utils import shuffle
from keras.optimizers import Adam,rmsprop
from keras.models import model_from_json, load_model
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D as Conv2D
from keras.layers.convolutional import Deconv2D as Conv2DTranspose
from keras.layers import Lambda, Input, Dense, MaxPooling2D, BatchNormalization,Input
from keras.layers import UpSampling2D, Dropout, Flatten, Reshape, RepeatVector, LeakyReLU,Activation
from keras.callbacks import ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.callbacks import EarlyStopping
np.random.seed(7)
from keras.callbacks import CSVLogger
from keras.callbacks import History 
import random
from itertools import product
import matplotlib.pyplot as plt
import prettytable
from prettytable import PrettyTable

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/gdrive/My Drive"

'A Decentralized Framework for Collision Free Multi-Robot Navigation.gslides'
'A Fog-Aware Adaptive Framework for Optimizing the Performance of Resource Constrained Self-driving Robots.gslides'
 AnomalyDetectionVAE
'Assurance Framework.pptx'
 Beta-VAE-New-Results.gslides
'B-VAE For Novelty Detection  .gslides'
 carla-different-weather.mp4
 carla-same-weather.mp4
'Car Supplies.gsheet'
'Change Point Detection for Perception Based Systems.gslides'
'Change Point Detection for Perception Based Systems.pptx'
'Colab Notebooks'
'Collaborators List-Dubey-FINAL.gsheet'
'Collaborators List-Dubey-FINAL.xlsx'
'DeepNNCar (1).gslides'
 DeepNNCar_2019
 DeepNNCar-Framework.gslides
 DeepNNCar.gslides
 DeepNNCar.pptx
'Designing Safety Cases for DeepNNCar.gdoc'
 DrawIO
'Emergency braking example.docx'
'Emergency braking example.gdoc'
'Experiment Section Carla.gdoc'
 frame3097.png
 Framework.gslides
'Framework to identify out-of-distribution images.gslides'
'Identifying Out-of distribution Images using VAE

In [0]:
# path to USB
USBPath = "/content/gdrive/My Drive/"
trainingFolders = ["sunny-train"]
history = CSVLogger(USBPath + '/kerasloss.csv', append=True, separator=';')

#Load complete input images without shuffling
def load_images(paths):
    numImages = 0
    inputs = []
    for path in paths:
        numFiles = len(glob.glob1(path,'*.png'))
        numImages += numFiles
        i=0
        for img in glob.glob(path+'*.png'):
            i+=1
            print(i)
            img = cv2.imread(img)
            img = cv2.resize(img, (200,66))
            img = img /255.
            inputs.append(img)
    #inpu = shuffle(inputs)
    print("Total number of images:%d" %(numImages))
    return inputs

def createFolderPaths(folders):
    paths = []
    for folder in folders:
        path = USBPath + folder + '/'
        paths.append(path)
    return paths

def load_training_images():
    paths = createFolderPaths(trainingFolders)
    return load_images(paths)

def load_steering_value():
    miny= -1
    maxy= 1
    Y=[]
    dataset = "/content/gdrive/My Drive/" + '/sunny-train.csv'
    with open(dataset, 'rt') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
            output=[]
            x=(float(row[0])-miny)/(maxy-miny)
            output.append(x)
            Y.append(output)
    return Y        

In [0]:
#Loading images from the datasets
image_input = load_training_images()
steer_output = load_steering_value()
print("Total Steering values:%d"%len(steer_output))
image_input = np.array(image_input)
steer_output = np.array(steer_output)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


KeyboardInterrupt: ignored

In [0]:
#CNN-VAE model.
def createModel():

    model = Sequential()
    input= Input(shape=(66,200,3), name='image')
    steer_inp = BatchNormalization(epsilon=0.001, axis=-1,momentum=0.99)(input)
    layer1 = Conv2D(24, (5, 5), padding="valid", strides=(2, 2), activation="relu")(steer_inp)
    layer2 = Conv2D(36, (5, 5), padding="valid", strides=(2, 2), activation="relu")(layer1)
    layer3 = Conv2D(48, (5, 5), padding="valid", strides=(2, 2), activation="relu")(layer2)
    layer4 = Conv2D(64, (3, 3), padding="valid", strides=(1, 1), activation="relu")(layer3)
    layer5 = Conv2D(64, (3, 3), padding="valid", strides=(1, 1), activation="relu")(layer4)
    layer6 = Flatten()(layer5)
    layer7 = Dense(1164, activation='relu')(layer6)
    layer8 = Dense(100, activation='relu')(layer7)
    layer9 = Dense(10, activation='relu')(layer8)
    steer_out = Dense(1, activation='tanh')(layer9)
    model=Model(inputs=input, outputs=steer_out)
  
    return model

def trainModel(model, X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05,random_state=42)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss="mse", metrics=['mae', 'acc'])
    #adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #model.compile(loss='mse', optimizer=adam)
    # checkpoint
    filePath = "weights.best.hdf5"
    callbacks_list = [
    ModelCheckpoint(filePath, monitor='val_mean_absolute_error', verbose=1, save_best_only=False),
    EarlyStopping(monitor='val_mean_absolute_error', patience=10, verbose=0),
    TensorBoard(log_dir='./tensorboard/', histogram_freq=0, write_graph=False, write_images=False)
    ]
#     checkpoint = ModelCheckpoint(filePath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#     callbacks_list = [checkpoint, history]
    model.fit(X_train, Y_train, epochs=10, batch_size=64,validation_data=(X_test, Y_test),callbacks=callbacks_list, verbose=2)
    

def saveModel(model):
	model_json = model.to_json()
	with open("model.json", "w") as json_file:
		json_file.write(model_json)

	model.save_weights("model.h5")
	print("Saved model to disk")


In [0]:
model = createModel()
print("create a new model")
trainModel(model, image_input, steer_output)
print("completed training the model")
saveModel(model)